In [4]:
model_type = "model_log_reg"

import sys
sys.path.append(rf"..\Classification\titanic\{model_type}")

import pandas as pd
import yaml
import pickle
import os
import json

In [7]:
yaml_path = rf"..\..\{model_type}\src\config.yaml"
with open(yaml_path, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

In [9]:
params = {
          'test_data': os.path.join(
              config['init_path'],
              config['processed_data']['path'], 
              config['processed_data']['test']),                 
          'pipe': os.path.join(
            config['init_path'],
            config['pipe_feat_eng']['path']), 
          'model': os.path.join(
              config['init_path'],
              config['model']['path']),
          'model_version': config['model']['model_version'],
          'reports': os.path.join(
            config['init_path'],
            config['save_reports']['path_reports']),
          'target':config['feat_selection_params']['target'],
          'cols_2_drop_model':config['model_selection']['cols_2_drop'],
          'cols_2_drop_feat_sel' : config['feat_selection_params']['cols_2_drop'],    
          'submission': os.path.join(
              config['init_path'],
              config['submission']['path'])
          }

In [10]:
df_test = pd.read_parquet(params['test_data'])    
y_test_id = df_test[['PassengerId']].copy()
    
df_test.drop(
        columns=params['cols_2_drop_feat_sel'], 
        inplace=True)

In [11]:
df_test

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Ticket_1p,Cabin_1p,Embarked_mod,FamilySize,IsAlone
0,3,male,34.5,0.0,0.0,330911,7.8292,None,Q,3,None,SQ,1,1
1,3,female,47.0,1.0,0.0,363272,7.0000,None,S,3,None,SQ,2,0
2,2,male,62.0,0.0,0.0,240276,9.6875,None,Q,2,None,SQ,1,1
3,3,male,27.0,0.0,0.0,315154,8.6625,None,S,3,None,SQ,1,1
4,3,female,22.0,1.0,1.0,3101298,12.2875,None,S,3,None,SQ,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,male,NaN,0.0,0.0,A53236,8.0500,None,S,A,None,SQ,1,1
414,1,female,39.0,0.0,0.0,PC17758,108.9000,C105,C,P,C,C,1,1
415,3,male,38.5,0.0,0.0,SOTONOQ3101262,7.2500,None,S,S,None,SQ,1,1
416,3,male,NaN,0.0,0.0,359309,8.0500,None,S,3,None,SQ,1,1


In [12]:
pipe_path = os.path.join(
        params['model'],
        f"feat_sel_pipe_{config['feat_selection_params']['pipe_version']}.pkl")

In [16]:
with open(pipe_path, "rb") as file:
        pipe = pickle.load(file)
    # appply pipe


In [17]:
pipe

,steps,"[('columntransformer', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('numerical_pipe_con', ...), ('numerical_pipe_equalwidth', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [ ]:
df_test_transf = pipe.transform(df_test)